In [1]:
"""
atividade_pratica_ix.py

Implementação completa para:
- Base Pima Indians Diabetes (baixada automaticamente)
- Preprocessamento: zeros -> mediana (atributos inválidos), StandardScaler
- Divisão 70% train / 30% test
- Modelos: SVM (SVC(probability=True)), MLPClassifier
- Otimização: GridSearchCV, RandomizedSearchCV (n_iter=20), Optuna (TPE), TPOT (genético)
- CV: KFold cv=5
- Métricas: accuracy, precision, recall, f1, roc_auc, cohen_kappa
- Plots: ROC, métricas comparativas, boxplots CV

Obs: Ajuste parâmetros de TPOT (gerations/population) conforme poder computacional.
"""
import os
import time
import warnings
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, confusion_matrix, roc_curve, cohen_kappa_score)
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.exceptions import ConvergenceWarning

# Optional heavier packages; may raise if not installed
try:
    import optuna
except Exception as e:
    optuna = None
    print("Optuna não disponível:", e)

try:
    from tpot import TPOTClassifier
except Exception as e:
    TPOTClassifier = None
    print("TPOT não disponível:", e)

from scipy.stats import loguniform
import joblib

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# ------------- 1) Carregar a base Pima (download do UCI) ----------------
DATA_URL = "https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv"
# alternativa: https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
# o arquivo referenciado contém colunas nomeadas

print("Carregando dataset Pima Indians Diabetes ...")
df = pd.read_csv(DATA_URL)
print("Dimensões:", df.shape)
print("Colunas:", df.columns.tolist())
# Verificar formato esperado: se estiver em formato sem nomes (9 colunas), adaptar
if df.shape[1] == 9 and 'Outcome' in df.columns:
    pass
elif df.shape[1] == 9 and 'Outcome' not in df.columns:
    df.columns = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','Outcome']
else:
    # Caso seja outro formato, tentar mapear por índice (fallback)
    expected = ['Pregnancies','Glucose','BloodPressure','SkinThickness','Insulin','BMI','DiabetesPedigreeFunction','Age','Outcome']
    if df.shape[1] == 9:
        df.columns = expected
    else:
        raise ValueError("Formato inesperado do CSV. Verifique o arquivo em DATA_URL")

# ------------- 1b) Preprocessamento: zeros -> mediana (atributos inválidos) ----
# Atributos que não podem ser zero: Glucose, BloodPressure, SkinThickness, Insulin, BMI
cols_where_zero_invalid = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
print("Contagem de zeros antes da imputação:\n", (df[cols_where_zero_invalid] == 0).sum())

# Substituir zeros por NaN, em seguida imputar pela mediana
df[cols_where_zero_invalid] = df[cols_where_zero_invalid].replace(0, np.nan)
imputer = SimpleImputer(strategy='median')
df[cols_where_zero_invalid] = imputer.fit_transform(df[cols_where_zero_invalid])

print("Contagem de NaNs após imputação (deve ser 0):\n", df.isna().sum())

# ------------- 1c) Separar X e y, treino/teste 70/30 -----------------------
X = df.drop(columns=['Outcome'])
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=RANDOM_STATE, stratify=y
)

print(f"Split: X_train {X_train.shape}, X_test {X_test.shape}")

# ------------- Pipeline comum: scaler ------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------- 2) Definir modelos e espaços de hiperparâmetros -------------
svm = SVC(probability=True, random_state=RANDOM_STATE)
mlp = MLPClassifier(random_state=RANDOM_STATE, early_stopping=True)

# Espaços solicitados
svm_param_grid = {
    'C': [1e-3, 1e-2, 1e-1, 1, 10, 100, 1e3],
    'kernel': ['linear', 'rbf'],
    'gamma': [1e-4, 1e-3, 1e-2, 1e-1, 1],  # usado apenas se kernel='rbf'
    'class_weight': [None, 'balanced']
}
svm_param_dist = {
    'C': loguniform(1e-3, 1e3),
    'kernel': ['linear', 'rbf'],
    'gamma': loguniform(1e-4, 1.0),
    'class_weight': [None, 'balanced']
}

mlp_param_grid = {
    'hidden_layer_sizes': [(64,), (128,), (64,32), (128,64), (256,128)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
    'learning_rate_init': [1e-4, 1e-3, 1e-2],
    'max_iter': [300, 600],
    'early_stopping': [True]
}
mlp_param_dist = {
    'hidden_layer_sizes': [(64,), (128,), (64,32), (128,64), (256,128)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': loguniform(1e-6, 1e-2),
    'learning_rate_init': loguniform(1e-4, 1e-2),
    'max_iter': [300, 600],
    'early_stopping': [True]
}

# ------------- helpers: métricas e execução de buscas ---------------------
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

def compute_metrics(clf, X_tr, y_tr, X_ts, y_ts):
    """Retorna dict com métricas no conjunto de teste e probas/roc"""
    y_pred = clf.predict(X_ts)
    if hasattr(clf, "predict_proba"):
        y_proba = clf.predict_proba(X_ts)[:, 1]
    else:
        # SVC with probability=True should have predict_proba; fallback decision_function
        try:
            y_proba = clf.decision_function(X_ts)
            # decision_function may produce scores; convert to [0,1] via minmax
            y_proba = (y_proba - y_proba.min()) / (y_proba.max() - y_proba.min() + 1e-9)
        except:
            y_proba = np.zeros_like(y_pred)
    metrics = {}
    metrics['accuracy'] = accuracy_score(y_ts, y_pred)
    metrics['precision'] = precision_score(y_ts, y_pred, zero_division=0)
    metrics['recall'] = recall_score(y_ts, y_pred, zero_division=0)
    metrics['f1'] = f1_score(y_ts, y_pred, zero_division=0)
    try:
        metrics['roc_auc'] = roc_auc_score(y_ts, y_proba)
    except ValueError:
        metrics['roc_auc'] = np.nan
    metrics['kappa'] = cohen_kappa_score(y_ts, y_pred)
    return metrics, y_proba

def cross_val_stats(estimator, X, y, scoring='accuracy', cv=kf):
    """Retorna média e std do cross_val_score"""
    scores = cross_val_score(estimator, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    return scores.mean(), scores.std(), scores

# ------------- 3) Estratégias de otimização por modelo --------------------
results = []  # armazena resultados por tuple (modelo, estrategia, best_estimator, best_params, cv_mean, cv_std, cv_scores, test_metrics, time)

def run_grid_search(name, estimator, param_grid, X_tr, y_tr):
    print(f"\n---- GridSearch: {name} ----")
    start = time.time()
    gs = GridSearchCV(estimator, param_grid, cv=kf, n_jobs=-1, scoring='accuracy', verbose=0)
    gs.fit(X_tr, y_tr)
    elapsed = time.time() - start
    mean_cv = gs.best_score_
    # obter cv scores para o melhor estimador (re-run cross_val_score)
    mean, std, scores = cross_val_stats(gs.best_estimator_, X_tr, y_tr, scoring='accuracy', cv=kf)
    print("Melhor score (CV) ~", mean_cv, "best_params:", gs.best_params_, "tempo(s):", elapsed)
    return gs.best_estimator_, gs.best_params_, mean, std, scores, elapsed

def run_random_search(name, estimator, param_dist, X_tr, y_tr, n_iter=20):
    print(f"\n---- RandomizedSearch (n_iter={n_iter}): {name} ----")
    start = time.time()
    rs = RandomizedSearchCV(estimator, param_dist, n_iter=n_iter, cv=kf, n_jobs=-1, scoring='accuracy', random_state=RANDOM_STATE, verbose=0)
    rs.fit(X_tr, y_tr)
    elapsed = time.time() - start
    mean, std, scores = cross_val_stats(rs.best_estimator_, X_tr, y_tr, scoring='accuracy', cv=kf)
    print("Melhor score (CV) ~", rs.best_score_, "best_params:", rs.best_params_, "tempo(s):", elapsed)
    return rs.best_estimator_, rs.best_params_, mean, std, scores, elapsed

def run_optuna_svm(X_tr, y_tr, n_trials=40):
    if optuna is None:
        raise RuntimeError("Optuna não instalado")
    print("\n---- Optuna (SVM) ----")
    def objective(trial):
        C = trial.suggest_loguniform('C', 1e-3, 1e3)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf'])
        class_weight = trial.suggest_categorical('class_weight', [None, 'balanced'])
        if kernel == 'rbf':
            gamma = trial.suggest_loguniform('gamma', 1e-4, 1.0)
        else:
            gamma = 'scale'
        clf = SVC(C=C, kernel=kernel, gamma=gamma, class_weight=class_weight, probability=True, random_state=RANDOM_STATE)
        scores = cross_val_score(clf, X_tr, y_tr, cv=kf, scoring='accuracy', n_jobs=-1)
        return scores.mean()
    sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    best = study.best_params
    print("Optuna best:", study.best_value, best)
    # construir estimador final com melhores params
    if best['kernel'] == 'rbf':
        gamma = best['gamma']
    else:
        gamma = 'scale'
    clf_best = SVC(C=best['C'], kernel=best['kernel'], gamma=gamma, class_weight=best['class_weight'], probability=True, random_state=RANDOM_STATE)
    clf_best.fit(X_tr, y_tr)
    mean, std, scores = cross_val_stats(clf_best, X_tr, y_tr, scoring='accuracy', cv=kf)
    return clf_best, best, mean, std, scores, study

def run_optuna_mlp(X_tr, y_tr, n_trials=40):
    if optuna is None:
        raise RuntimeError("Optuna não instalado")
    print("\n---- Optuna (MLP) ----")
    def objective(trial):
        hidden = trial.suggest_categorical('hidden_layer_sizes', [(64,), (128,), (64,32), (128,64), (256,128)])
        activation = trial.suggest_categorical('activation', ['relu','tanh'])
        solver = trial.suggest_categorical('solver', ['adam','lbfgs'])
        alpha = trial.suggest_loguniform('alpha', 1e-6, 1e-2)
        lr = trial.suggest_loguniform('learning_rate_init', 1e-4, 1e-2)
        max_it = trial.suggest_categorical('max_iter', [300,600])
        clf = MLPClassifier(hidden_layer_sizes=hidden, activation=activation, solver=solver, alpha=alpha,
                            learning_rate_init=lr, max_iter=max_it, early_stopping=True, random_state=RANDOM_STATE)
        scores = cross_val_score(clf, X_tr, y_tr, cv=kf, scoring='accuracy', n_jobs=-1)
        return scores.mean()
    sampler = optuna.samplers.TPESampler(seed=RANDOM_STATE)
    study = optuna.create_study(direction='maximize', sampler=sampler)
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    best = study.best_params
    print("Optuna best:", study.best_value, best)
    clf_best = MLPClassifier(hidden_layer_sizes=best['hidden_layer_sizes'], activation=best['activation'],
                             solver=best['solver'], alpha=best['alpha'], learning_rate_init=best['learning_rate_init'],
                             max_iter=best['max_iter'], early_stopping=True, random_state=RANDOM_STATE)
    clf_best.fit(X_tr, y_tr)
    mean, std, scores = cross_val_stats(clf_best, X_tr, y_tr, scoring='accuracy', cv=kf)
    return clf_best, best, mean, std, scores, study

def run_tpot(name, X_tr, y_tr, generations=5, population_size=20, scoring='accuracy'):
    if TPOTClassifier is None:
        raise RuntimeError("TPOT não instalado")
    print(f"\n---- TPOT ({name}) generations={generations}, population_size={population_size} ----")
    start = time.time()
    tpot = TPOTClassifier(generations=generations, population_size=population_size, random_state=RANDOM_STATE, n_jobs=1)
    tpot.fit(X_tr, y_tr)
    elapsed = time.time() - start
    print("TPOT best pipeline exported")
    best_pipeline = tpot.fitted_pipeline_
    mean, std, scores = cross_val_stats(best_pipeline, X_tr, y_tr, scoring='accuracy', cv=kf)
    return best_pipeline, {'tpot': 'pipeline'}, mean, std, scores, elapsed, tpot

# ------------- 4) Executar as 4 estratégias para SVM e MLP ------------------
models = [
    ('SVM', SVC(probability=True, random_state=RANDOM_STATE), svm_param_grid, svm_param_dist),
    ('MLP', MLPClassifier(random_state=RANDOM_STATE, early_stopping=True), mlp_param_grid, mlp_param_dist)
]

for model_name, estimator, param_grid, param_dist in models:
    # Re-criar estimator fresh
    print("\n\n############################################################")
    print("MODELO:", model_name)
    print("############################################################")
    # Grid Search
    try:
        best_est, best_params, mean_cv, std_cv, scores_cv, t_elapsed = run_grid_search(model_name, estimator, param_grid, X_train_scaled, y_train)
        test_metrics, test_proba = compute_metrics(best_est, X_train_scaled, y_train, X_test_scaled, y_test)
        results.append({
            'model': model_name,
            'strategy': 'GridSearch',
            'estimator': best_est,
            'best_params': best_params,
            'cv_mean': mean_cv,
            'cv_std': std_cv,
            'cv_scores': scores_cv,
            'test_metrics': test_metrics,
            'time_s': t_elapsed
        })
    except Exception as e:
        print("GridSearch falhou:", e)

    # Random Search (n_iter=20)
    try:
        best_est, best_params, mean_cv, std_cv, scores_cv, t_elapsed = run_random_search(model_name, estimator, param_dist, X_train_scaled, y_train, n_iter=20)
        test_metrics, test_proba = compute_metrics(best_est, X_train_scaled, y_train, X_test_scaled, y_test)
        results.append({
            'model': model_name,
            'strategy': 'RandomSearch',
            'estimator': best_est,
            'best_params': best_params,
            'cv_mean': mean_cv,
            'cv_std': std_cv,
            'cv_scores': scores_cv,
            'test_metrics': test_metrics,
            'time_s': t_elapsed
        })
    except Exception as e:
        print("RandomSearch falhou:", e)

    # Optuna (Bayesiana via TPE)
    try:
        if model_name == 'SVM':
            best_est, best_params, mean_cv, std_cv, scores_cv, study = run_optuna_svm(X_train_scaled, y_train, n_trials=40)
        else:
            best_est, best_params, mean_cv, std_cv, scores_cv, study = run_optuna_mlp(X_train_scaled, y_train, n_trials=40)
        test_metrics, test_proba = compute_metrics(best_est, X_train_scaled, y_train, X_test_scaled, y_test)
        results.append({
            'model': model_name,
            'strategy': 'Optuna(TPE)',
            'estimator': best_est,
            'best_params': best_params,
            'cv_mean': mean_cv,
            'cv_std': std_cv,
            'cv_scores': scores_cv,
            'test_metrics': test_metrics,
            'time_s': None,
            'optuna_study': None
        })
    except Exception as e:
        print("Optuna falhou:", e)

    # TPOT (Genético) - reduzir gerações/população se máquina fraca
    try:
        tpot_generations = 5
        tpot_population_size = 20
        best_pipeline, best_params, mean_cv, std_cv, scores_cv, t_elapsed, tpot_obj = run_tpot(model_name, X_train_scaled, y_train, generations=tpot_generations, population_size=tpot_population_size)
        test_metrics, test_proba = compute_metrics(best_pipeline, X_train_scaled, y_train, X_test_scaled, y_test)
        results.append({
            'model': model_name,
            'strategy': 'TPOT(Genetic)',
            'estimator': best_pipeline,
            'best_params': best_params,
            'cv_mean': mean_cv,
            'cv_std': std_cv,
            'cv_scores': scores_cv,
            'test_metrics': test_metrics,
            'time_s': t_elapsed,
            'tpot_obj': tpot_obj
        })
    except Exception as e:
        print("TPOT falhou:", e)

# ------------- 5) Agrupar resultados e salvar/exportar ---------------------
print("\n\n===== RESUMO DOS RESULTADOS =====")
summary_rows = []
for r in results:
    print(f"{r['model']} | {r['strategy']} | CV mean={r['cv_mean']:.4f} ± {r['cv_std']:.4f} | Test acc={r['test_metrics']['accuracy']:.4f} | time(s)={r.get('time_s')}")
    row = {
        'model': r['model'],
        'strategy': r['strategy'],
        'cv_mean': r['cv_mean'],
        'cv_std': r['cv_std'],
        'test_accuracy': r['test_metrics']['accuracy'],
        'test_precision': r['test_metrics']['precision'],
        'test_recall': r['test_metrics']['recall'],
        'test_f1': r['test_metrics']['f1'],
        'test_roc_auc': r['test_metrics']['roc_auc'],
        'test_kappa': r['test_metrics']['kappa'],
        'time_s': r.get('time_s')
    }
    summary_rows.append(row)

df_summary = pd.DataFrame(summary_rows)
df_summary.to_csv("resumo_resultados.csv", index=False)
print("Resumo salvo em resumo_resultados.csv")

# ------------- 6) Plots: comparações de métricas e ROC ---------------------
os.makedirs("figs", exist_ok=True)

# a) Bar plot comparando accuracy/test_f1/roc_auc por estratégia e modelo
metrics_to_plot = ['test_accuracy', 'test_precision', 'test_recall', 'test_f1', 'test_roc_auc']
plt.figure(figsize=(12,6))
df_plot = df_summary.melt(id_vars=['model','strategy'], value_vars=metrics_to_plot, var_name='metric', value_name='value')
sns.barplot(data=df_plot, x='metric', y='value', hue='model', ci='sd')
plt.title("Comparação de métricas (por modelo) — média no conjunto de teste")
plt.ylim(0,1)
plt.legend(title='Modelo')
plt.tight_layout()
plt.savefig("figs/comparacao_metricas_modelo.png", dpi=200)
print("Figura salva: figs/comparacao_metricas_modelo.png")
plt.close()

# b) Boxplot dos scores de CV por estratégia (acurácia)
plt.figure(figsize=(12,6))
box_data = []
labels = []
for r in results:
    if 'cv_scores' in r and r['cv_scores'] is not None:
        box_data.append(r['cv_scores'])
        labels.append(f"{r['model']}\n{r['strategy']}")
if box_data:
    plt.boxplot(box_data, labels=labels, vert=True)
    plt.xticks(rotation=45, ha='right')
    plt.ylabel("Acurácia (CV folds)")
    plt.title("Distribuição dos folds (K=5) por estratégia/modelo")
    plt.tight_layout()
    plt.savefig("figs/boxplot_cv_scores.png", dpi=200)
    print("Figura salva: figs/boxplot_cv_scores.png")
    plt.close()

# c) ROC Curves: plotar para os melhores por (modelo, estratégia) — se possível
plt.figure(figsize=(10,8))
for r in results:
    est = r['estimator']
    try:
        _, y_proba = compute_metrics(est, X_train_scaled, y_train, X_test_scaled, y_test)
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        auc = r['test_metrics']['roc_auc']
        plt.plot(fpr, tpr, label=f"{r['model']} | {r['strategy']} (AUC={auc:.3f})")
    except Exception as e:
        # p. ex. se não é possível obter probas
        continue
plt.plot([0,1],[0,1],'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves — Test set")
plt.legend(loc='lower right', fontsize='small')
plt.tight_layout()
plt.savefig("figs/roc_all.png", dpi=200)
print("Figura salva: figs/roc_all.png")
plt.close()

# ------------- 7) Salvar objetos (joblib) --------------------------------
os.makedirs("models", exist_ok=True)
for i, r in enumerate(results):
    fname = f"models/{r['model']}_{r['strategy']}.joblib"
    try:
        joblib.dump(r['estimator'], fname)
        print("Salvo modelo:", fname)
    except Exception as e:
        print("Falha ao salvar modelo:", fname, e)

# ------------- 8) Relatório resumido em CSV e prints -----------------------
print("\nArquivo resumo_resultados.csv gerado. Figuras em ./figs . Modelos em ./models")
print(df_summary)

# ------------- 9) Análises rápidas (respostas às questões) -------------
print("\n=== Análise comparativa rápida (automática) ===")
# Melhor estratégia por modelo (baseado em cv_mean)
for model_name in df_summary['model'].unique():
    dfm = df_summary[df_summary['model'] == model_name]
    best_row = dfm.loc[dfm['cv_mean'].idxmax()]
    print(f"Para {model_name}, melhor estratégia (CV mean) = {best_row['strategy']} (cv_mean={best_row['cv_mean']:.4f}, test_acc={best_row['test_accuracy']:.4f})")

print("\nVerifique 'resumo_resultados.csv' para detalhes e as figuras em 'figs/'.")
print("Fim do script.")


/home/iagora/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Carregando dataset Pima Indians Diabetes ...
Dimensões: (768, 9)
Colunas: ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
Contagem de zeros antes da imputação:
 Glucose            5
BloodPressure     35
SkinThickness    227
Insulin          374
BMI               11
dtype: int64
Contagem de NaNs após imputação (deve ser 0):
 Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64
Split: X_train (537, 8), X_test (231, 8)


############################################################
MODELO: SVM
############################################################

---- GridSearch: SVM ----
Melhor score (CV) ~ 0.786067843544479 best_params: {'C': 1, 'class_weight': None, 'gamma': 0.01, 'kernel': 'rb

[I 2025-11-20 15:32:45,221] A new study created in memory with name: no-name-1d34397f-b903-423e-882c-632c5c15f0ce


Melhor score (CV) ~ 0.7841640706126687 best_params: {'C': np.float64(0.3905441275210791), 'class_weight': None, 'gamma': np.float64(0.012563152773938664), 'kernel': 'linear'} tempo(s): 0.9620873928070068

---- Optuna (SVM) ----


  0%|          | 0/40 [00:00<?, ?it/s]/tmp/ipykernel_2512/1185693727.py:209: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-3, 1e3)
Best trial: 0. Best value: 0.784164:   0%|          | 0/40 [00:00<?, ?it/s]

...

TPOT best pipeline exported


===== RESUMO DOS RESULTADOS =====
SVM | GridSearch | CV mean=0.7861 ± 0.0623 | Test acc=0.7359 | time(s)=77.64752125740051
SVM | RandomSearch | CV mean=0.7842 ± 0.0463 | Test acc=0.7273 | time(s)=0.9620873928070068
SVM | Optuna(TPE) | CV mean=0.7860 ± 0.0458 | Test acc=0.7273 | time(s)=None
SVM | TPOT(Genetic) | CV mean=0.7563 ± 0.0700 | Test acc=0.7532 | time(s)=878.9393124580383
MLP | GridSearch | CV mean=0.7897 ± 0.0423 | Test acc=0.7489 | time(s)=155.48255705833435
MLP | RandomSearch | CV mean=0.7823 ± 0.0464 | Test acc=0.7489 | time(s)=6.930534601211548
MLP | Optuna(TPE) | CV mean=0.7916 ± 0.0422 | Test acc=0.7532 | time(s)=None
MLP | TPOT(Genetic) | CV mean=0.7563 ± 0.0700 | Test acc=0.7532 | time(s)=903.8761239051819
Resumo salvo em resumo_resultados.csv


/tmp/ipykernel_2512/1185693727.py:395: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar='sd'` for the same effect.

  sns.barplot(data=df_plot, x='metric', y='value', hue='model', ci='sd')


Figura salva: figs/comparacao_metricas_modelo.png


/tmp/ipykernel_2512/1185693727.py:413: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  plt.boxplot(box_data, labels=labels, vert=True)


Figura salva: figs/boxplot_cv_scores.png
Figura salva: figs/roc_all.png
Salvo modelo: models/SVM_GridSearch.joblib
Salvo modelo: models/SVM_RandomSearch.joblib
Salvo modelo: models/SVM_Optuna(TPE).joblib
Salvo modelo: models/SVM_TPOT(Genetic).joblib
Salvo modelo: models/MLP_GridSearch.joblib
Salvo modelo: models/MLP_RandomSearch.joblib
Salvo modelo: models/MLP_Optuna(TPE).joblib
Salvo modelo: models/MLP_TPOT(Genetic).joblib

Arquivo resumo_resultados.csv gerado. Figuras em ./figs . Modelos em ./models
  model       strategy   cv_mean    cv_std  test_accuracy  test_precision  \
0   SVM     GridSearch  0.786068  0.062308       0.735931        0.672414   
1   SVM   RandomSearch  0.784164  0.046323       0.727273        0.655172   
2   SVM    Optuna(TPE)  0.786016  0.045843       0.727273        0.655172   
3   SVM  TPOT(Genetic)  0.756317  0.069959       0.753247        0.639535   
4   MLP     GridSearch  0.789720  0.042271       0.748918        0.645570   
5   MLP   RandomSearch  0.78229